# HDAWG

Some text here ...



In [1]:
import zhinst.toolkit as tk

And initialize the device. We use the `tk.HDAWG` instrument driver, its device serial is `"dev8030"` and we connect via ethernet (`"1gbe"`) to the host at `"10.42.0.226"`.

In [2]:
hdawg = tk.HDAWG("hdawg1", "dev8030", interface="1gbe", host="10.42.0.226")
hdawg.setup()           # set up data server connection
hdawg.connect_device()  # connect device to data server

Successfully connected to data server at 10.42.0.2268004 api version: 6
Successfully connected to device DEV8030 on interface 1GBE


## AWG Cores 

besides the `nodetree` property, the devices also have other properties, e.g. `AWG`s. The HDAWG has a list of four AWGs.

In [4]:
hdawg.awgs

[hdawg1: <zhinst.toolkit.control.drivers.hdawg.AWG object at 0x000002C815106160>
     parent  : <zhinst.toolkit.control.drivers.hdawg.HDAWG object at 0x000002C8150F8F98>
     index   : 0
     sequence: 
            type: None
             ('target', <DeviceTypes.HDAWG: 'hdawg'>)
             ('clock_rate', 2400000000.0)
             ('period', 0.0001)
             ('trigger_mode', 'None')
             ('repetitions', 1)
             ('alignment', 'End with Trigger')
             ('n_HW_loop', 1)
             ('dead_time', 5e-06)
             ('trigger_delay', 0)
             ('latency', 1.6e-07)
             ('trigger_cmd_1', '//\n')
             ('trigger_cmd_2', '//\n')
             ('wait_cycles', 0)
             ('dead_cycles', 0)
             ('reset_phase', False)
       IQ Modulation DISABLED,
 hdawg1: <zhinst.toolkit.control.drivers.hdawg.AWG object at 0x000002C815106358>
     parent  : <zhinst.toolkit.control.drivers.hdawg.HDAWG object at 0x000002C8150F8F98>
     index   : 1
 

The awgs feature a list of `Parameters` and methods that allow for a high-level control of the AWG Core and the pulse sequence that is programmed on it.  

The parameters include 

* `output1`
* `output2`
* `gain1`
* `gain2`
* `modulation_freq`
* ...

In [7]:
import time

# different ways of turning outputs on or off
for awg in hdawg.awgs:
    awg.outputs(("on", "on"))
    time.sleep(0.5)
    awg.output1("off")
    time.sleep(0.5)
    awg.output2("off")
    time.sleep(0.5)

In [9]:
# also the AWGs have parameters
hdawg.awgs[0].modulation_freq

Node: oscs/0/freq
Description: Sets the modulation frequency of the AWG output channels.
Type: Double
Properties: Read, Write
Unit: Hz

In [10]:
# enable IQ modulation, i.e. appling the neccessary settings
hdawg.awgs[0].enable_iq_modulation()

In [13]:
import numpy as np

# a simple sweep
for f in np.linspace(10e6, 20e6, 101):
    hdawg.awgs[0].modulation_freq(f)
    time.sleep(0.1)

### Programming the AWG sequencer

In [16]:
# the AWG has a sequence program object
hd.awgs[0]._program

In [17]:
# we can get and set parameters of the sequence program
hd.awgs[0].sequence_params

{'sequence_type': None,
 'sequence_parameters': {'target': 'hdawg',
  'clock_rate': 2400000000.0,
  'period': 0.0001,
  'trigger_mode': 'None',
  'repetitions': 1,
  'alignment': 'End with Trigger',
  'n_HW_loop': 1,
  'dead_time': 5e-06,
  'trigger_delay': 0,
  'latency': 1.6e-07,
  'trigger_cmd_1': '//',
  'trigger_cmd_2': '//',
  'wait_cycles': 0,
  'dead_cycles': 0}}

In [ ]:
# use the awg core as a master trigger
hd.awgs[0].set_sequence_params(
    sequence_type="Trigger",
    period=100e-6,
    repetitions=100,
)
hd.awgs[0].compile()

In [18]:
# upload waveforms as arrays in "Simple" mode
hd.awgs[0].set_sequence_params(
    sequence_type="Simple",
    period=100e-6,
    repetitions=100,
    alignment="End with Trigger",
    trigger_mode="Send Trigger",
)

In [19]:
hd.awgs[0]

hd1: <zhinst.toolkit.hdawg.AWG object at 0x0000021E467D3320>
    parent  : <zhinst.toolkit.hdawg.HDAWG object at 0x0000021E467D3198>
    index   : 0
    sequence: 
           type: Simple
            ('target', 'hdawg')
            ('clock_rate', 2400000000.0)
            ('period', 0.0001)
            ('trigger_mode', 'Send Trigger')
            ('repetitions', 100)
            ('alignment', 'End with Trigger')
            ('n_HW_loop', 1)
            ('dead_time', 5e-06)
            ('trigger_delay', 0)
            ('latency', 1.6e-07)
            ('trigger_cmd_1', 'setTrigger(1);\n')
            ('trigger_cmd_2', 'setTrigger(0);\n')
            ('wait_cycles', 28500)
            ('dead_cycles', 1500)
            ('buffer_lengths', [800])
      IQ Modulation ENABLED:
         frequency   : 19999999.999998864
         phase_shift : 90.0
         gains       : 1.0, 1.0

In [20]:
# for this 'Simple' sequence we can queue up arbitrary waveforms 
hd.awgs[0].queue_waveform(np.ones(1000), -np.ones(1000))

Current length of queue: 1


In [21]:
# and also reset the queue
hd.awgs[0].reset_queue()

In [22]:
# let's queue up some waveforms
for amp in np.linspace(-1, 1, 10):
    hd.awgs[0].queue_waveform(amp*np.ones(400), -amp*np.ones(400))

Current length of queue: 1
Current length of queue: 2
Current length of queue: 3
Current length of queue: 4
Current length of queue: 5
Current length of queue: 6
Current length of queue: 7
Current length of queue: 8
Current length of queue: 9
Current length of queue: 10


In [25]:
# the waveforms are all in the queue now
hd.awgs[0].waveforms

In [26]:
# now we have to compile the sequence program and then we can upload all the waveforms
hd.awgs[0].compile()
hd.awgs[0].upload_waveforms()

# we could also just use hd.awgs[0].compile_and_upload_waveforms()

Compilation successful
hd1-0: Sequencer status: ELF file uploaded
Upload of 10 waveforms took 0.020005226135253906 s


In [27]:
# let's run the sequence
hd.awgs[0].run()

Started AWG hd1-0!


In [28]:
# is it still running?
hd.awgs[0].is_running

0

In [31]:
# we can also just replace a single one of the waveforms with a waveform of the same length
hd.awgs[0].replace_waveform(np.random.rand(400), np.random.rand(400), i=4)
hd.awgs[0].upload_waveforms()

# check the waveform in the LabOne UI! We do not need to recompile as long as the waveform is of the same length.

Upload of 10 waveforms took 0.018970489501953125 s
